<h1 style="text-align: center;">🥈Transformação de dados da camada bronze para silver</h1>


<img src="Imagem_projeto.png" alt="Visualização do Projeto" width="600" style="display: block; margin-left: auto; margin-right: auto;"/>


**📄 Transformações padrão**

+ Valores Nulos;
    - Padronizar valores nulos que foram carregados como **("NaN", "nan", "None", "none", "NULL", "null", "")**

+ Higienização simples de strings:
    - Remover sujeiras como espaços extras, quebras de linha e caracteres de controle invisíveis

+ Analise de IDs duplicados;
    - Verificação simples de IDs que podem estar duplicados na camada bronze e caso esteja, sera mantido apenas o ultimo id com base no scrape_id ou ultima data


In [1]:
#📦import das principais libs


import pandas as pd
import numpy as np
import re, io, csv
from typing import List, Optional, Sequence
import psycopg2
from psycopg2.extras import execute_values
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
import geopandas as gpd
from shapely.geometry import Point
from urllib.parse import urlparse

In [2]:
#import das variaveis de ambiente
load_dotenv(r"..\scripts\.env")


#conexao com postgres
try:
    DB_URI = (f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}")


    engine = create_engine(DB_URI)
    conn = engine.connect()
    raw_conn = engine.raw_connection()
    print("✅ Conexao realizada com sucesso!")
except Exception as e:
    print(f"❌ Erro ao conectar ao banco de dados {e}")


✅ Conexao realizada com sucesso!


### **Tratamento tabela T_DIM_ANUNCIO**

In [63]:
#Leitura da tabela Anuncio
df_anuncio = pd.read_sql('SELECT * FROM a_bronze. "T_DIM_ANUNCIO"', conn)
df_anuncio.head(3)

,id_anuncio,listing_url,scrape_id,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,picture_url
0,14063,https://www.airbnb.com/rooms/14063,20180414160018,Living in a Postcard,"Besides the most iconic's view, our apartment ...",NaN,"Besides the most iconic's view, our apartment ...",none,Best and favorite neighborhood of Rio. Perfect...,NaN,Everything is there. METRO is 5 min walk. Dir...,NaN,NaN,strictly no smoking in the apartment ! We want...,https://a0.muscache.com/im/pictures/66421/ae9b...
1,17878,https://www.airbnb.com/rooms/17878,20180414160018,Very Nice 2Br - Copacabana - WiFi,Please note that special rates apply for New Y...,- large balcony which looks out on pedestrian ...,Please note that special rates apply for New Y...,none,This is the best spot in Rio. Everything happe...,NaN,Excellent location. Close to all major public ...,The entire apartment is yours. It is a vacatio...,I will be available throughout your stay shoul...,Please leave the apartment in a clean fashion ...,https://a0.muscache.com/im/pictures/65320518/3...
2,24480,https://www.airbnb.com/rooms/24480,20180414160018,Nice and cozy near Ipanema Beach,My studio is located in the best of Ipanema. ...,The studio is located at Vinicius de Moraes St...,My studio is located in the best of Ipanema. ...,none,"The beach, the lagoon, Ipanema is a great loca...","O prédio é bastante simples , mas o apartament...",NaN,"From the International airport, take a regula...",Os hóspedes podem perguntar por email suas que...,Please remove sand when you come from the beac...,https://a0.muscache.com/im/pictures/11955612/b...


In [38]:
#Analise de formato de colunas
df_anuncio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70185 entries, 0 to 70184
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id_anuncio             70185 non-null  int64 
 1   listing_url            70185 non-null  object
 2   scrape_id              70185 non-null  int64 
 3   name                   70185 non-null  object
 4   summary                70185 non-null  object
 5   space                  70185 non-null  object
 6   description            70185 non-null  object
 7   experiences_offered    70185 non-null  object
 8   neighborhood_overview  70185 non-null  object
 9   notes                  70185 non-null  object
 10  transit                70185 non-null  object
 11  access                 70185 non-null  object
 12  interaction            70185 non-null  object
 13  house_rules            70185 non-null  object
 14  picture_url            70185 non-null  object
dtypes: int64(2), object

In [57]:
#Tratamento de valores nulos
valores_nulos = ["NaN", "nan", "None", "none", "NULL", "null", ""]

contagem_nulos = df_anuncio.apply(lambda col: col.isnull().sum() + col.isin(valores_nulos).sum())

#Criando df para avaliar o percentual de valores nulos por coluna.
nulos_anuncio = pd.DataFrame({
    "colunas" : contagem_nulos.index,
    "total_nulos": contagem_nulos.values,
    "%nulos" : (contagem_nulos.values / len(df_anuncio) * 100).round(2).astype(str) + "%"
})

nulos_anuncio

,colunas,total_nulos,%nulos
0,id_anuncio,0,0.0%
1,listing_url,0,0.0%
2,scrape_id,0,0.0%
3,name,83,0.12%
4,summary,4943,7.04%
5,space,30096,42.88%
6,description,3031,4.32%
7,experiences_offered,70185,100.0%
8,neighborhood_overview,34418,49.04%
9,notes,51079,72.78%


#### Notas tabela Anuncio:

+ Colunas com mais de 60% de valores nulo serão removidos da camada Silver.
+ Padronizar urls dos anuncios
+ Padronizar strings
+ Criar colunas de auditoria

#### Aplicar transformações da tabela "T_DIM_ANUNCIO" e ETL para o banco na camada silver

In [79]:
#Copia do df para padronização
df = df_anuncio.copy()

#1) Padronização dos valores nulos
valores_nulos = ["NaN", "nan", "None", "none", "NULL", "null", ""]
colunas = df.select_dtypes(include="object").columns
for c in colunas:
    df[c] = df[c].replace(list(valores_nulos), np.nan)


#2) Higienização simples de strings
def clean_text(s):
    if not isinstance(s, str): return s
    s = s.strip()
    s = re.sub(r"\s+", " ", s) # colapsa espacos 
    s = "".join(ch for ch in s if ch >= " ") #Remover caracteres de controle \n \t etc..
    return s 

for c in colunas:
    df[c] = df[c].map(clean_text)


#4) Remover colunas >60% de valores nulos
null_ratio = df.isnull().mean()
to_drop = null_ratio[null_ratio >= 0.6].index.to_list()
df = df.drop(columns=to_drop, errors='ignore')


#5) Verificação de IDs duplicados e remoção do ultimo scraped_id

df = (df
      .sort_values(['id_anuncio', 'scrape_id'], ascending=[True, False])
      .drop_duplicates(subset=['id_anuncio'], keep='first'))


#6) validação de URL
def is_valid_url(u):
    if not isinstance(u, str) or not u: return False
    try:
        p = urlparse(u)
        return bool(p.scheme and p.netloc)
    except:
        return False
    
for ucol in ["listing_url","picture_url"]:
    if ucol in df.columns:
        df.loc[~df[ucol].map(is_valid_url), ucol] = np.nan


#7) Auditoria
df["fonte"] = 'Airbnb_bronze'
df['dt_ingestao'] = pd.Timestamp.utcnow()


#8) df com os ajustes realizados "Pronto para inserir no banco"


df_anuncio_silver = df.copy()

C:\Users\User\AppData\Local\Temp\ipykernel_17276\3782966486.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] = df[c].replace(list(valores_nulos), np.nan)


In [80]:
df_anuncio_silver

,id_anuncio,listing_url,scrape_id,name,summary,space,description,neighborhood_overview,transit,access,interaction,house_rules,picture_url,fonte,dt_ingestao
0,14063,https://www.airbnb.com/rooms/14063,20180414160018,Living in a Postcard,"Besides the most iconic's view, our apartment ...",NaN,"Besides the most iconic's view, our apartment ...",Best and favorite neighborhood of Rio. Perfect...,Everything is there. METRO is 5 min walk. Dire...,NaN,NaN,strictly no smoking in the apartment ! We want...,https://a0.muscache.com/im/pictures/66421/ae9b...,Airbnb_bronze,2025-08-17 21:58:12.064897+00:00
1,17878,https://www.airbnb.com/rooms/17878,20180414160018,Very Nice 2Br - Copacabana - WiFi,Please note that special rates apply for New Y...,- large balcony which looks out on pedestrian ...,Please note that special rates apply for New Y...,This is the best spot in Rio. Everything happe...,Excellent location. Close to all major public ...,The entire apartment is yours. It is a vacatio...,I will be available throughout your stay shoul...,Please leave the apartment in a clean fashion ...,https://a0.muscache.com/im/pictures/65320518/3...,Airbnb_bronze,2025-08-17 21:58:12.064897+00:00
49801,21280,https://www.airbnb.com/rooms/21280,20200420135919,Renovated Modern Apt. Near Beach,Immaculately renovated top-floor apartment ove...,Immaculately renovated top-floor apartment in ...,Immaculately renovated top-floor apartment ove...,This is the best neighborhood in Zona Sul. For...,The new metro station is just a few steps away...,"This is an older ""Art Deco"" style building, so...",Someone will be there at check in and check ou...,This is a booking agreement for rental of a tw...,https://a0.muscache.com/im/pictures/60851312/b...,Airbnb_bronze,2025-08-17 21:58:12.064897+00:00
2,24480,https://www.airbnb.com/rooms/24480,20180414160018,Nice and cozy near Ipanema Beach,My studio is located in the best of Ipanema. T...,The studio is located at Vinicius de Moraes St...,My studio is located in the best of Ipanema. T...,"The beach, the lagoon, Ipanema is a great loca...",NaN,"From the International airport, take a regular...",Os hóspedes podem perguntar por email suas que...,Please remove sand when you come from the beac...,https://a0.muscache.com/im/pictures/11955612/b...,Airbnb_bronze,2025-08-17 21:58:12.064897+00:00
3,25026,https://www.airbnb.com/rooms/25026,20180414160018,Beautiful Modern Decorated Studio in Copa,"Our apartment is a little gem, everyone loves ...",This newly renovated studio (last renovations ...,"Our apartment is a little gem, everyone loves ...",Copacabana is a lively neighborhood and the ap...,At night we recommend you to take taxis only. ...,"internet wi-fi, cable tv, air cond, ceiling fa...","Only at check in, we like to leave our guests ...",Smoking outside only. Family building so pleas...,https://a0.muscache.com/im/pictures/3003965/68...,Airbnb_bronze,2025-08-17 21:58:12.064897+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68584,43514675,https://www.airbnb.com/rooms/43514675,20200524171540,Quarto no Leblon,Meu apartamento é simples e muito aconchegante...,NaN,Meu apartamento é simples e muito aconchegante...,NaN,NaN,NaN,NaN,NaN,https://a0.muscache.com/im/pictures/74ed68fe-b...,Airbnb_bronze,2025-08-17 21:58:12.064897+00:00
68585,43515526,https://www.airbnb.com/rooms/43515526,20200524171540,"Lindo Apartamento, 2 quadras da praia",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://a0.muscache.com/im/pictures/3dce639d-a...,Airbnb_bronze,2025-08-17 21:58:12.064897+00:00
68586,43517044,https://www.airbnb.com/rooms/43517044,20200524171540,Conforto e concentração pra sua estadia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://a0.muscache.com/im/pictures/80589a65-9...,Airbnb_bronze,2025-08-17 21:58:12.064897+00:00
68587,43522964,https://www.airbnb.com/rooms/43522964,20200524171540,Botafogo ll,Excelente apartamento 2 quartos e dois banheir...,NaN,Excelente apartamento 2 quartos e dois banheir...,NaN,NaN,NaN,NaN,NaN,https://a0.muscach

#### Realizar carga no banco da tabela b_silver."T_DIM_ANUNCIO" 

In [32]:
#ETL PARA TABELA SILVER

def _ensure_columns(df: pd.DataFrame, target_cols: Sequence[str]) -> pd.DataFrame:
    """Garante que o DF tenha todas as colunas do contrato e na ordem correta."""
    for c in target_cols:
        if c not in df.columns:
            df[c] = np.nan
    return df[list(target_cols)].copy()


def _nan_to_none_records(df: pd.DataFrame) -> List[tuple]:
    """Converte NaN -> None (NULL no Postgres) e retorna lista de tuplas linha-a-linha."""
    records = []
    for row in df.itertuples(index=False, name=None):
        # transforma cada valor NaN em None
        fixed = tuple(None if (isinstance(x, float) and np.isnan(x)) else x for x in row)
        records.append(fixed)
    return records


def upsert_df(
    conn,
    df: pd.DataFrame,
    schema: str,
    table: str,
    target_cols: Sequence[str],
    conflict_cols: Sequence[str],
    update_cols: Optional[Sequence[str]] = None,
    page_size: int = 10000,
):
    """
    Faz UPSERT de um DF para uma tabela Postgres.
    - target_cols: colunas na ordem do contrato (iguais à tabela).
    - conflict_cols: colunas da UNIQUE/PK usadas no ON CONFLICT.
    - update_cols: quais colunas atualizar em caso de conflito.
      (por padrão, todas as target_cols EXCETO as de conflito).
    """
    if not len(df):
        return  # nada a fazer

    # 1) Garante contrato (colunas + ordem)
    df_up = _ensure_columns(df, target_cols)

    # 2) Define colunas a atualizar no DO UPDATE
    if update_cols is None:
        update_cols = [c for c in target_cols if c not in conflict_cols]
    if not update_cols:
        # Se não sobrou nada para atualizar, faça DO NOTHING para evitar erro
        on_conflict_sql = f'ON CONFLICT ({", ".join(f"""\"{c}\"""" for c in conflict_cols)}) DO NOTHING'
    else:
        set_sql = ", ".join(f'"{c}"=EXCLUDED."{c}"' for c in update_cols)
        on_conflict_sql = (
            f'ON CONFLICT ({", ".join(f"""\"{c}\"""" for c in conflict_cols)}) DO UPDATE SET {set_sql}'
        )

    # 3) Monta o INSERT com placeholder %s (execute_values preenche em lote)
    cols_sql = ", ".join(f'"{c}"' for c in target_cols)
    sql = f'INSERT INTO {schema}."{table}" ({cols_sql}) VALUES %s {on_conflict_sql};'

    # 4) Converte NaN->None e envia em lotes
    records = _nan_to_none_records(df_up)
    with conn.cursor() as cur:
        execute_values(cur, sql, records, page_size=page_size)
    conn.commit()

In [89]:

TARGET_COLS = [
    "id_anuncio","listing_url","scrape_id","name","summary","space","description",
    "neighborhood_overview","transit","access","interaction","house_rules",
    "picture_url","fonte","dt_ingestao"
]


upsert_df(
    conn=raw_conn,  
    df=df_anuncio_silver,
    schema="b_silver",
    table="T_DIM_ANUNCIO",
    target_cols=TARGET_COLS,
    conflict_cols=["id_anuncio"],   # chave de UPSERT (PK/UNIQUE)
    # update_cols=None  -> atualiza todas as colunas exceto a de conflito (default)
    page_size=10000
)
print("✅ T_DIM_ANUNCIO carregada/atualizada no Silver.")

✅ T_DIM_ANUNCIO carregada/atualizada no Silver.


### **Tratamento tabela T_DIM_ANFITRIAO**

In [3]:
#leitura da tabela anfitrião

df_anfitriao = pd.read_sql('SELECT * FROM  a_bronze. "T_DIM_ANFITRIAO"', conn)
df_anfitriao.head(3)

,host_id,fk_anuncio,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified
0,53598,14063,https://www.airbnb.com/users/show/53598,Shalev,2009-11-12,FL,"Hello , my name is Shalev , I am an orchestra ...",NaN,NaN,f,https://a0.muscache.com/im/users/53598/profile...,https://a0.muscache.com/im/users/53598/profile...,Botafogo,1.0,1.0,"['email', 'phone', 'reviews', 'jumio']",t,t
1,68997,17878,https://www.airbnb.com/users/show/68997,Matthias,2010-01-08,"Rio de Janeiro, Rio de Janeiro, Brazil",I used to work as a journalist all around the ...,within an hour,100%,t,https://a0.muscache.com/im/pictures/67b13cea-8...,https://a0.muscache.com/im/pictures/67b13cea-8...,Copacabana,2.0,2.0,"['email', 'phone', 'reviews']",t,f
2,99249,24480,https://www.airbnb.com/users/show/99249,Goya,2010-03-26,"Rio de Janeiro, Rio de Janeiro, Brazil",Welcome to Rio!\r\nI am a filmmaker and a tea...,within an hour,100%,f,https://a0.muscache.com/im/pictures/6b40475c-2...,https://a0.muscache.com/im/pictures/6b40475c-2...,Ipanema,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,t


##### Analise inicial, tipo de dados, valores nulos, novas features.

In [4]:
df_anfitriao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43180 entries, 0 to 43179
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   host_id                    43180 non-null  int64  
 1   fk_anuncio                 43180 non-null  int64  
 2   host_url                   43180 non-null  object 
 3   host_name                  43180 non-null  object 
 4   host_since                 43147 non-null  object 
 5   host_location              43180 non-null  object 
 6   host_about                 43180 non-null  object 
 7   host_response_time         43180 non-null  object 
 8   host_response_rate         43180 non-null  object 
 9   host_is_superhost          43180 non-null  object 
 10  host_thumbnail_url         43180 non-null  object 
 11  host_picture_url           43180 non-null  object 
 12  host_neighbourhood         43180 non-null  object 
 13  host_listings_count        43147 non-null  flo

In [5]:
#Tratamento de valores nulos
valores_nulos = ["NaN", "nan", "None", "none", "NULL", "null", ""]

contagem_nulos = df_anfitriao.apply(lambda col: col.isnull().sum() + col.isin(valores_nulos).sum())

#Criando df para avaliar o percentual de valores nulos por coluna.
nulos_anfitriao = pd.DataFrame({
    "colunas" : contagem_nulos.index,
    "total_nulos": contagem_nulos.values,
    "%nulos" : (contagem_nulos.values / len(df_anfitriao) * 100).round(2).astype(str) + "%"
})
nulos_anfitriao

,colunas,total_nulos,%nulos
0,host_id,0,0.0%
1,fk_anuncio,0,0.0%
2,host_url,0,0.0%
3,host_name,33,0.08%
4,host_since,33,0.08%
5,host_location,332,0.77%
6,host_about,27467,63.61%
7,host_response_time,19886,46.05%
8,host_response_rate,19886,46.05%
9,host_is_superhost,33,0.08%


In [6]:
df_anfitriao['host_total_listings_count'].sort_values(ascending=False)

38681    1495.0
42570     336.0
38634     327.0
38636     325.0
38633     313.0
          ...  
28037       NaN
28825       NaN
40584       NaN
40996       NaN
41908       NaN
Name: host_total_listings_count, Length: 43180, dtype: float64

In [7]:
# contagem por host_id
dup_counts = df_anfitriao.groupby("host_id").size().sort_values(ascending=False)

n_hosts = dup_counts.shape[0]
n_rows  = len(df_anfitriao)
n_dups  = (dup_counts > 1).sum()

print(f"linhas={n_rows:,}  hosts únicos={n_hosts:,}  hosts com duplicata={n_dups:,}")
dup_counts[dup_counts > 1].head(10)

linhas=43,180  hosts únicos=43,180  hosts com duplicata=0


Series([], dtype: int64)

#### Notas:
+ Coluna **host_since**         : Esta como obj mas é data
+ Coluna **host_response_rate** : Esta como obj mas é uma coluna de percentual
+ Coluna **host_about**         : Sera exluida por ter mais de 60% de valores nulos e nao ter muita relevancia para o contesto de analise
+ Coluna **host_listings_count**: Esta como float porem é uma coluna que recebe valores interios, **(Sera necessario arrumar o tipo de dados no banco)**
+ Coluna **host_total_listings_count** : Esta como float porem é uma coluna que recebe valores interios
+ Colunas **"host_is_superhost","host_has_profile_pic","host_identity_verified"**, transformar t/f em True e False

In [8]:
#Copia do df para padronização
df = df_anfitriao.copy()

#1) Padronização dos valores nulos
valores_nulos = ["NaN", "nan", "None", "none", "NULL", "null", ""]
colunas = df.select_dtypes(include="object").columns
for c in colunas:
    df[c] = df[c].replace(list(valores_nulos), np.nan)


#2) Higienização simples de strings
def clean_text(s):
    if not isinstance(s, str): return s
    s = s.strip()
    s = re.sub(r"\s+", " ", s) # colapsa espacos 
    s = "".join(ch for ch in s if ch >= " ") #Remover caracteres de controle \n \t etc..
    return s 

for c in colunas:
    df[c] = df[c].map(clean_text)


#4) Remover colunas >60% de valores nulos
null_ratio = df.isnull().mean()
to_drop = null_ratio[null_ratio >= 0.6].index.to_list()
df = df.drop(columns=to_drop, errors='ignore')



#5)transformar coluna obj em data
df['host_since'] = pd.to_datetime(df['host_since'], errors='coerce')
df['host_since'] = df['host_since'].apply(lambda x: x.to_pydatetime() if pd.notnull(x) else None)
df['host_since'] = df['host_since'].fillna(pd.Timestamp("1900-01-01"))


#6)Tratar strings dentro da coluna host_response_rate (%) e tranformar strinsgs de valores nulos para 0 e manter o dado em proporcao
df['host_response_rate'] = (df['host_response_rate']
                                      .astype(str)
                                      .str.strip()
                                      .str.replace('%','', regex=False)
                                      .replace(valores_nulos, 0)
                                      .astype(float)
                                      )/100


#7) Correção de dados das coluna host_listings_count e host_total_listings_count de float para inteiro

fltcolumns = ['host_listings_count', 'host_total_listings_count']

for col in fltcolumns:
    df[col] = (pd.to_numeric(df[col], errors='coerce')
                         .fillna(0)
                         .astype(int)
                         )

#8) Transformar booleanas t/f em True e False
bool_map = {"t": True, "f": False, True: True, False: False}
for c in ["host_is_superhost","host_has_profile_pic","host_identity_verified"]:
    if c in df.columns:
        df[c] = df[c].map(bool_map).astype("bool")




#9) Padronizar categorias
map_resp_time = {
    "within an hour":"within an hour",
    "within a few hours":"within a few hours",
    "within a day":"within a day",
    "a few days or more":"a few days or more"
}
df["host_response_time"] = df["host_response_time"].str.strip().str.lower().map(map_resp_time)



#10) Padronizar Urls
for ucol in ["host_url","host_picture_url","host_thumbnail_url"]:
    if ucol in df.columns:
        df.loc[~df[ucol].map(is_valid_url), ucol] = np.nan


#11)host_verifications: transformar string/lista-serializada em array “limpo” (se vier como texto tipo ['email','phone'])

import ast
def parse_verifs(v):
    if isinstance(v, list): return v
    if not isinstance(v, str) or not v.strip(): return None
    try:
        x = ast.literal_eval(v)
        if isinstance(x, list):
            return [str(i).strip().lower() for i in x]
    except:
        pass
    return None

df["host_verifications"] = df["host_verifications"].apply(parse_verifs)


#12) Auditoria
df["fonte"] = 'Airbnb_bronze'
df['dt_ingestao'] = pd.Timestamp.utcnow()



#13) df com os ajustes realizados "Pronto para inserir no banco"
df_anfitriao_silver = df.copy()



NameError: name 'is_valid_url' is not defined

#### Realizar carga no banco da tabela b_silver."T_ANFITRIAO" 

In [122]:

TARGET_COLS = ['host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_response_time', 'host_response_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'fonte',
       'dt_ingestao'
]


upsert_df(
    conn=raw_conn,  
    df=df_anfitriao_silver,
    schema="b_silver",
    table="T_DIM_ANFITRIAO",
    target_cols=TARGET_COLS,
    conflict_cols=["host_id"],   # chave de UPSERT (PK/UNIQUE)
    # update_cols=None  -> atualiza todas as colunas exceto a de conflito (default)
    page_size=10000
)
print("✅ T_DIM_ANFITRIAO carregada/atualizada no Silver.")

✅ T_DIM_ANUNCIO carregada/atualizada no Silver.


### **Tratamento tabela T_DIM_LOCALIZACAO**

In [48]:
#leitura tabela localização

df_localizacao = pd.read_sql('SELECT * FROM a_bronze. "T_DIM_LOCALIZACAO"', conn)
df_localizacao.head()

,id_localizacao,fk_anuncio,street,neighbourhood,neighbourhood_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact
0,1,14063,"Rio de Janeiro, RJ, Brazil",Botafogo,Botafogo,Rio de Janeiro,RJ,22250-040,Rio De Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,-22.946854,-43.182737,t
1,2,17878,"Rio de Janeiro, Rio de Janeiro, Brazil",Copacabana,Copacabana,Rio de Janeiro,Rio de Janeiro,22020-050,Rio De Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,-22.965919,-43.178962,t
2,3,24480,"Rio de Janeiro, Rio de Janeiro, Brazil",Ipanema,Ipanema,Rio de Janeiro,Rio de Janeiro,22411-010,Rio De Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,-22.985698,-43.201935,t
3,4,25026,"Rio de Janeiro, Rio de Janeiro, Brazil",Copacabana,Copacabana,Rio de Janeiro,Rio de Janeiro,22060-020,Rio De Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,-22.977117,-43.190454,t
4,5,31560,"Rio de Janeiro, RJ, Brazil",Ipanema,Ipanema,Rio de Janeiro,RJ,22410-003,Rio De Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,-22.983024,-43.214270,t


In [49]:
df_localizacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 902210 entries, 0 to 902209
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id_localizacao          902210 non-null  int64  
 1   fk_anuncio              902210 non-null  int64  
 2   street                  902210 non-null  object 
 3   neighbourhood           902210 non-null  object 
 4   neighbourhood_cleansed  902210 non-null  object 
 5   city                    902210 non-null  object 
 6   state                   902210 non-null  object 
 7   zipcode                 902210 non-null  object 
 8   market                  902210 non-null  object 
 9   smart_location          902210 non-null  object 
 10  country_code            902210 non-null  object 
 11  country                 902210 non-null  object 
 12  latitude                902210 non-null  float64
 13  longitude               902210 non-null  float64
 14  is_location_exact   

#### Notas:

+ Coluna **neighbourhood** : Esta coluna recebe o valor inserido pelo anfitrião e depois é ajustada pelo airbnb na coluna **neighbourhood_cleansed** com isso irei eliminar a coluna neighbourhood e vou renomear a coluna neighbourhood_cleansed para neighbourhood
+ Coluna **smart_location**



In [50]:
#Usando API REST publica para obter os limites de bairros do rio de janeiro.
url_bairros = "https://pgeo3.rio.rj.gov.br/arcgis/rest/services/Cartografia/Limites_administrativos/MapServer/4/query?where=1=1&outFields=*&f=geojson"
bairros_rio = gpd.read_file(url_bairros)

In [51]:
# 1) Garante CRS e cria GeoDataFrame de pontos (lon, lat)
gdf_pts = gpd.GeoDataFrame(
    df_localizacao.copy(),
    geometry=gpd.points_from_xy(df_localizacao['longitude'], df_localizacao['latitude']),
    crs="EPSG:4326"
)

# 2) Garante que bairros_rio está no mesmo CRS
if bairros_rio.crs is None:
    bairros_rio = bairros_rio.set_crs("EPSG:4326")
elif bairros_rio.crs != gdf_pts.crs:
    bairros_rio = bairros_rio.to_crs(gdf_pts.crs)

# 3) sjoin (ponto dentro do polígono do bairro)
gjoined = gpd.sjoin(
    gdf_pts,
    bairros_rio[['nome', 'regiao_adm', 'rp', 'geometry']],  # pegue só o que precisa
    how="left",
    predicate="within"
).drop(columns=['index_right'])

In [52]:
# Padronização de campos

def title_pt(s: pd.Series) -> pd.Series:
    def fix(x):
        if not isinstance(x, str): return x
        t = x.title()
        for a,b in [(" De "," de "),(" Da "," da "),(" Do "," do "),
                    (" Dos "," dos "),(" Das "," das "),(" Em "," em "),(" E "," e ")]:
            t = t.replace(a,b)
        return t.strip()
    return s.apply(fix)

# Bairro oficial
gjoined['neighbourhood_cleansed'] = title_pt(gjoined['nome'])

# City e Market: se caiu em bairro do RJ → “Rio de Janeiro”
gjoined['city'] = gjoined['city'].where(gjoined['neighbourhood_cleansed'].isna(), 'Rio de Janeiro')
gjoined['market'] = gjoined['market'].where(gjoined['neighbourhood_cleansed'].isna(), 'Rio de Janeiro')

# Country (corrige scrapes errados tipo “Andorra”)
gjoined['country_code'] = gjoined.get('country_code', pd.Series(index=gjoined.index))
gjoined['country'] = gjoined.get('country', pd.Series(index=gjoined.index))
gjoined.loc[gjoined['city'].eq('Rio de Janeiro'), ['country_code','country']] = ['BR','Brazil']

# Fallback por bbox do Brasil (opcional)
mask_bbox_br = gjoined['longitude'].between(-74, -34) & gjoined['latitude'].between(-34, 5)
gjoined.loc[mask_bbox_br, ['country_code','country']] = ['BR','Brazil']

# smart_location = "Cidade, País"
gjoined['smart_location'] = (
    title_pt(gjoined['city']).fillna('') + ', ' + title_pt(gjoined['country']).fillna('')
).str.strip(', ')

# Padroniza capitalização
gjoined['city'] = title_pt(gjoined['city'])
gjoined['market'] = title_pt(gjoined['market'])

In [53]:
# Remove geometria para voltar a DataFrame “puro”
df_localizacao_silver = pd.DataFrame(gjoined.drop(columns='geometry'))

log_geo = {
    "pct_bairro_atribuido": round(df_localizacao_silver['neighbourhood_cleansed'].notna().mean()*100, 2),
    "pct_city_atribuido": round(df_localizacao_silver['city'].notna().mean()*100, 2),
    "corrigidos_para_BR": int(((df_localizacao_silver['country_code'] == 'BR') & (df_localizacao.get('country_code') != 'BR')).sum()),
    "total_linhas": len(df_localizacao_silver),
}
print(log_geo)

{'pct_bairro_atribuido': np.float64(99.96), 'pct_city_atribuido': np.float64(100.0), 'corrigidos_para_BR': 31, 'total_linhas': 902210}


In [54]:
df_localizacao_silver.head()

,id_localizacao,fk_anuncio,street,neighbourhood,neighbourhood_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,nome,regiao_adm,rp
0,1,14063,"Rio de Janeiro, RJ, Brazil",Botafogo,Botafogo,Rio de Janeiro,RJ,22250-040,Rio de Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,-22.946854,-43.182737,t,Botafogo,BOTAFOGO,Zona Sul
1,2,17878,"Rio de Janeiro, Rio de Janeiro, Brazil",Copacabana,Copacabana,Rio de Janeiro,Rio de Janeiro,22020-050,Rio de Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,-22.965919,-43.178962,t,Copacabana,COPACABANA,Zona Sul
2,3,24480,"Rio de Janeiro, Rio de Janeiro, Brazil",Ipanema,Ipanema,Rio de Janeiro,Rio de Janeiro,22411-010,Rio de Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,-22.985698,-43.201935,t,Ipanema,LAGOA,Zona Sul
3,4,25026,"Rio de Janeiro, Rio de Janeiro, Brazil",Copacabana,Copacabana,Rio de Janeiro,Rio de Janeiro,22060-020,Rio de Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,-22.977117,-43.190454,t,Copacabana,COPACABANA,Zona Sul
4,5,31560,"Rio de Janeiro, RJ, Brazil",Ipanema,Ipanema,Rio de Janeiro,RJ,22410-003,Rio de Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,-22.983024,-43.214270,t,Ipanema,LAGOA,Zona Sul


In [56]:
df_localizacao_silver['market'].value_counts()

#Removendo dois valores que nao conseguimos mapear com o geopandas
df_localizacao_silver = df_localizacao_silver[df_localizacao_silver['market'] != 'Other (International)']

In [57]:
df_localizacao_silver.drop(columns=['neighbourhood','neighbourhood_cleansed'], inplace=True)
df_localizacao_silver.rename(columns={'nome': 'neighbourhood'}, inplace=True)

In [59]:
#Definicao de pontos turisticos
pois = pd.DataFrame({
    'poi': [
        'Cristo Redentor','Pao de Acucar','Praia de Copacabana','Praia de Ipanema',
        'Maracana','Arcos da Lapa','Museu do Amanha','Jardim Botanico'
    ],
    'lat': [-22.951916, -22.948611, -22.971177, -22.986869, -22.912161, -22.912167, -22.895911, -22.968801],
    'lon': [-43.210487, -43.156389, -43.182543, -43.155444, -43.230184, -43.179954, -43.180763, -43.223593]
})


gdf_pts = gpd.GeoDataFrame(
    df_localizacao_silver.copy(),
    geometry=gpd.points_from_xy(df_localizacao_silver['longitude'], df_localizacao_silver['latitude']),
    crs="EPSG:4326"
)

gdf_pois = gpd.GeoDataFrame(
    pois.copy(),
    geometry=gpd.points_from_xy(pois['lon'], pois['lat']),
    crs="EPSG:4326"
)

# Projeta para metros (UTM 23S)
gdf_pts_m = gdf_pts.to_crs(31983)
gdf_pois_m = gdf_pois.to_crs(31983)

# Distância para cada POI em km
for _, r in gdf_pois_m.iterrows():
    col = f"dist_{r['poi'].lower().replace(' ','_')}_km"
    gdf_pts_m[col] = gdf_pts_m.geometry.distance(r.geometry) / 1000.0

# POI mais próximo (usando as colunas calculadas)
dist_cols = [c for c in gdf_pts_m.columns if c.startswith('dist_') and c.endswith('_km')]
gdf_pts_m['nearest_poi_km'] = gdf_pts_m[dist_cols].min(axis=1)
gdf_pts_m['nearest_poi_name'] = gdf_pts_m[dist_cols].idxmin(axis=1).str.replace(r'^dist_|_km$', '', regex=True)

# Volta a DF se quiser
df_localizacao_silver = pd.DataFrame(gdf_pts_m.drop(columns='geometry'))

#Auditoria
df_localizacao_silver["fonte"] = 'Airbnb_bronze'
df_localizacao_silver['dt_ingestao'] = pd.Timestamp.utcnow()


In [60]:
df_localizacao_silver

,id_localizacao,fk_anuncio,street,city,state,zipcode,market,smart_location,country_code,country,...,dist_praia_de_copacabana_km,dist_praia_de_ipanema_km,dist_maracana_km,dist_arcos_da_lapa_km,dist_museu_do_amanha_km,dist_jardim_botanico_km,nearest_poi_km,nearest_poi_name,fonte,dt_ingestao
0,1,14063,"Rio de Janeiro, RJ, Brazil",Rio de Janeiro,RJ,22250-040,Rio de Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,...,2.693764,5.241443,6.200742,3.852050,5.645376,4.843959,2.693764,praia_de_copacabana,Airbnb_bronze,2025-08-18 23:18:09.678300+00:00
1,2,17878,"Rio de Janeiro, Rio de Janeiro, Brazil",Rio de Janeiro,Rio de Janeiro,22020-050,Rio de Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,...,0.688432,3.346509,7.940186,5.953718,7.755335,4.587990,0.688432,praia_de_copacabana,Airbnb_bronze,2025-08-18 23:18:09.678300+00:00
2,3,24480,"Rio de Janeiro, Rio de Janeiro, Brazil",Rio de Janeiro,Rio de Janeiro,22411-010,Rio de Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,...,2.557381,4.768766,8.643863,8.449579,10.177916,2.904091,2.557381,praia_de_copacabana,Airbnb_bronze,2025-08-18 23:18:09.678300+00:00
3,4,25026,"Rio de Janeiro, Rio de Janeiro, Brazil",Rio de Janeiro,Rio de Janeiro,22060-020,Rio de Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,...,1.044435,3.748849,8.267550,7.273149,9.048024,3.520814,1.044435,praia_de_copacabana,Airbnb_bronze,2025-08-18 23:18:09.678300+00:00
4,5,31560,"Rio de Janeiro, RJ, Brazil",Rio de Janeiro,RJ,22410-003,Rio de Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,...,3.507942,6.046838,8.015621,8.600274,10.241277,1.842540,1.842540,jardim_botanico,Airbnb_bronze,2025-08-18 23:18:09.678300+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902205,902206,38844730,"Barra da Tijuca, Rio de Janeiro, Brazil",Rio de Janeiro,Rio de Janeiro,22631,Rio de Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,...,16.813255,19.297708,15.332097,19.509718,20.431023,12.805599,12.805599,jardim_botanico,Airbnb_bronze,2025-08-18 23:18:09.678300+00:00
902206,902207,38846408,"Jacarepaguá, Rio de Janeiro, Brazil",Rio de Janeiro,Rio de Janeiro,22775-150,Rio de Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,...,21.633093,24.507441,17.799041,22.713228,23.179303,17.419460,17.419460,jardim_botanico,Airbnb_bronze,2025-08-18 23:18:09.678300+00:00
902207,902208,38846703,"Copacabana, Rio de Janeiro, Brazil",Rio de Janeiro,Rio de Janeiro,22011-040,Rio de Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,...,1.238424,3.431484,7.874035,5.546374,7.347693,4.985795,1.238424,praia_de_copacabana,Airbnb_bronze,2025-08-18 23:18:09.678300+00:00
902208,902209,38847050,"Barra da Tijuca, Rio de Janeiro, Brazil",Rio de Janeiro,Rio de Janeiro,22631-280,Rio de Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,...,16.716212,19.195891,15.269537,19.432877,20.360232,12.713099,12.713099,jardim_botanico,Airbnb_bronze,2025-08-18 23:18:09.678300+00:00


**Remover linhas onde FK_anuncio esteja duplicado**

In [63]:
df = df_localizacao_silver.copy()

# score de completude (quantos campos não-nulos)
cols_para_completude = [
    'street','city','state','zipcode','market','smart_location',
    'country_code','country','latitude','longitude','is_location_exact',
    'neighbourhood','regiao_adm','rp',
    'dist_cristo_redentor_km','dist_pao_de_acucar_km','dist_praia_de_copacabana_km',
    'dist_praia_de_ipanema_km','dist_maracana_km','dist_arcos_da_lapa_km',
    'dist_museu_do_amanha_km','dist_jardim_botanico_km',
    'nearest_poi_km','nearest_poi_name'
]
df['_nn'] = df[cols_para_completude].notna().sum(axis=1)

df['_has_ll'] = df['latitude'].notna() & df['longitude'].notna()

df = df.sort_values(
    by=['fk_anuncio','_nn','_has_ll','nearest_poi_km','dt_ingestao'],
    ascending=[True, False, False, True, False]
)

# dropa duplicatas mantendo a "melhor" (primeira do sort)
df_dedup = df.drop_duplicates(subset=['fk_anuncio'], keep='first').drop(columns=['_nn','_has_ll'])

# (opcional) sanity check
dups = df_dedup.duplicated(subset=['fk_anuncio']).sum()
assert dups == 0, f'Ainda há duplicatas em fk_anuncio ({dups}).'

df_localizacao_silver = df_dedup.copy()

In [81]:
# mapa BR: nome por extenso -> UF
UF_MAP = {
    "acre":"AC","alagoas":"AL","amapá":"AP","amapa":"AP","amazonas":"AM","bahia":"BA",
    "ceará":"CE","ceara":"CE","distrito federal":"DF","espírito santo":"ES","espirito santo":"ES",
    "goiás":"GO","goias":"GO","maranhão":"MA","maranhao":"MA","mato grosso":"MT","mato grosso do sul":"MS",
    "minas gerais":"MG","pará":"PA","para":"PA","paraíba":"PB","paraiba":"PB","paraná":"PR","parana":"PR",
    "pernambuco":"PE","piauí":"PI","piaui":"PI","rio de janeiro":"RJ","rio grande do norte":"RN",
    "rio grande do sul":"RS","rondônia":"RO","rondonia":"RO","roraima":"RR","santa catarina":"SC",
    "são paulo":"SP","sao paulo":"SP","sergipe":"SE","tocantins":"TO"
}
UF_SET = set(UF_MAP.values())

# regex para capturar sigla UF (dois caracteres A-Z entre separadores)
UF_RE = re.compile(r'(?<![A-Z])([A-Z]{2})(?![A-Z])')

def pick_uf_from_text(txt: str):
    """Tenta achar uma UF em texto (sigla ou nome por extenso)."""
    if not isinstance(txt, str) or not txt.strip():
        return None
    s = txt.strip()

    # 1) procurar sigla (ex.: ", RJ," ou "RJ -")
    m = UF_RE.search(s.upper())
    if m and m.group(1) in UF_SET:
        return m.group(1)

    # 2) procurar nome por extenso
    low = s.lower()
    # normaliza acentos básicos pro match simples
    low_norm = (low
        .replace('á','a').replace('ã','a').replace('â','a')
        .replace('é','e').replace('ê','e')
        .replace('í','i')
        .replace('ó','o').replace('õ','o').replace('ô','o')
        .replace('ú','u')
        .replace('ç','c')
    )
    for name, uf in UF_MAP.items():
        name_norm = (name
            .replace('á','a').replace('ã','a').replace('â','a')
            .replace('é','e').replace('ê','e')
            .replace('í','i')
            .replace('ó','o').replace('õ','o').replace('ô','o')
            .replace('ú','u')
            .replace('ç','c')
        )
        if name_norm in low_norm:
            return uf

    return None

def to_uf_row(row):
    """
    Resolve 'state' para UF:
    - se já é UF válida -> retorna
    - senão, tenta extrair de state, depois city, smart_location e street
    - se BR e ainda não achou, fallback: primeiras 2 letras de city (pouco recomendável, mas cumpre CHECK)
    - se não BR, limita a 3 chars maiúsculos (cumpre CHECK)
    """
    cc = str(row.get('country_code', '') or '').strip().upper()
    candidates = [
        row.get('state', None),
        row.get('city', None),
        row.get('smart_location', None),
        row.get('street', None),
    ]
    # já é UF?
    st = str(row.get('state', '') or '').strip().upper()
    if st in UF_SET:
        return st

    # tenta extrair de cada campo
    for cand in candidates:
        uf = pick_uf_from_text(str(cand) if cand is not None else '')
        if uf:
            return uf

    if cc == 'BR':
        # último recurso BR: se city tem "rio de janeiro", mapear
        uf = pick_uf_from_text(str(row.get('city', '') or ''))
        if uf:
            return uf
        # fallback bem conservador: nenhuma UF achada → None (para você tratar depois)
        return None
    else:
        # países não-BR: apenas garantir <=3 chars (ex.: 'NY', 'BC', etc.)
        s = str(row.get('state', '') or '').strip().upper()
        if not s:
            return None
        return s[:3]

# aplica
df_localizacao_silver['state'] = df_localizacao_silver.apply(to_uf_row, axis=1)

# sanity check antes do upsert: ninguém com >3 chars
viol = df_localizacao_silver['state'].dropna().astype(str).str.len() > 3
assert not viol.any(), f"States >3 chars ainda encontrados:\n{df_localizacao_silver.loc[viol, ['fk_anuncio','state','city']].head()}"

In [84]:
raw_conn.rollback()

In [87]:
TARGET_COLS = [
    'fk_anuncio',
    'street','city','state','zipcode','market','smart_location',
    'country_code','country','latitude','longitude','is_location_exact',
    'neighbourhood','regiao_adm','rp',
    'dist_cristo_redentor_km','dist_pao_de_acucar_km','dist_praia_de_copacabana_km',
    'dist_praia_de_ipanema_km','dist_maracana_km','dist_arcos_da_lapa_km',
    'dist_museu_do_amanha_km','dist_jardim_botanico_km',
    'nearest_poi_km','nearest_poi_name',
    'fonte'  # dt_ingestao fica de fora para usar DEFAULT
]

upsert_df(
    conn=raw_conn,  
    df=df_localizacao_silver,
    schema="b_silver",
    table="T_DIM_LOCALIZACAO",
    target_cols=TARGET_COLS,
    conflict_cols=["fk_anuncio"],   # chave de UPSERT (PK/UNIQUE)
    # update_cols=None  -> atualiza todas as colunas exceto a de conflito (default)
    page_size=10000
)
print("✅ T_DIM_LOCALIZACAO carregada/atualizada no Silver.")


✅ T_DIM_LOCALIZACAO carregada/atualizada no Silver.


### **Tratamento tabela T_DIM_PROPRIEDADE**

In [5]:
#Leitura da tabela propriedade
df_propriedade = pd.read_sql('SELECT * FROM a_bronze. "T_DIM_PROPRIEDADE"',conn)
df_propriedade.head(5)

,id_propriedade,fk_anuncio,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365
0,1,14063,Apartment,Entire home/apt,4,1.0,0.0,2.0,Real Bed,"{TV,Internet,""Air conditioning"",Kitchen,Doorma...",60.0,365.0,7 weeks ago,t,28.0,58.0,88.0,363.0
1,2,17878,Condominium,Entire home/apt,5,1.0,2.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",4.0,90.0,yesterday,t,11.0,29.0,58.0,286.0
2,3,24480,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Wifi,""Air conditioning"",""First ...",3.0,90.0,5 weeks ago,t,0.0,0.0,0.0,0.0
3,4,25026,Apartment,Entire home/apt,3,1.0,1.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",4.0,30.0,today,t,28.0,58.0,88.0,363.0
4,5,31560,Apartment,Entire home/apt,3,1.0,1.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",2.0,1125.0,5 weeks ago,t,15.0,45.0,75.0,345.0


#### Notas:

+ Coluna: **"beds"** faz sentido ser um valor decimal, sendo 1.5 = um banheiro + lavabo

+ Conlunas: **"bedrooms", "minimum_nights", "maximum_nights","availability_30","availability_60", "availability_90", "availability_365", "beds"**: Estão como float porem devem ser numeros inteiros

+ Colunas: obj serão normalizadas 

In [6]:
# =========================
# 0) Base
# =========================
df = df_propriedade.copy()

# =========================
# 1) Padronização de nulos + limpeza básica de strings
# =========================
VALORES_NULOS = {"NaN", "nan", "None", "none", "NULL", "null", ""}

obj_cols = df.select_dtypes(include="object").columns.tolist()

def clean_text(s):
    if not isinstance(s, str):
        return s
    s2 = s.strip()
    s2 = re.sub(r"\s+", " ", s2)         # colapsa espaços
    s2 = "".join(ch for ch in s2 if ch >= " ")  # remove \n \t etc.
    # normaliza marcadores de nulo
    return (None if s2.lower() in VALORES_NULOS else s2)

for c in obj_cols:
    df[c] = df[c].map(clean_text)

# =========================
# 2)dropar colunas >60% nulas:
# =========================
null_ratio = df.isnull().mean()
to_drop = null_ratio[ (null_ratio >= 0.60) & (null_ratio.index != "amenities") ].index.tolist()
df.drop(columns=to_drop, errors='ignore', inplace=True)

# =========================
# 3) Tipagem numérica
# =========================
# inteiros
int_cols = [
    "accommodates","bedrooms","beds",
    "minimum_nights","maximum_nights",
    "availability_30","availability_60","availability_90","availability_365"
]
for c in int_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype("int64")

# bathrooms com 1 casa decimal
if "bathrooms" in df.columns:
    df["bathrooms"] = pd.to_numeric(df["bathrooms"], errors="coerce").round(1)

# =========================
# 4) Booleanos
# =========================
# preserve <NA> -> vira None no insert
def to_bool_or_na(v):
    # nulos
    try:
        if v is None or pd.isna(v):
            return pd.NA
    except Exception:
        pass

    # já bool
    if isinstance(v, (bool, np.bool_)):
        return bool(v)

    # numérico 0/1
    if isinstance(v, (int, np.integer, float, np.floating)):
        if pd.isna(v): return pd.NA
        return bool(int(v))

    # string
    if isinstance(v, str):
        s = v.strip().lower()
        if s in {"t","true","1","y","yes","sim"}:
            return True
        if s in {"f","false","0","n","no","não","nao"}:
            return False
        if s in {"", "nan", "none", "null"}:
            return pd.NA

    # fallback: trata como NA (ou mude para False se preferir)
    return pd.NA

if "has_availability" in df.columns:
    df["has_availability"] = df["has_availability"].apply(to_bool_or_na).astype("boolean")

# =========================
# 5) Amenities: parse robusto + count + flags
# =========================
def parse_amenities(raw):
    """
    Converte strings como:
      '{TV,Internet,"Air conditioning",Kitchen,...}'
    em lista ['TV','Internet','Air conditioning','Kitchen',...]
    Usa csv para respeitar aspas/virgulas internas.
    """
    if raw is None or (isinstance(raw, float) and pd.isna(raw)):
        return []
    if isinstance(raw, list):
        # já é lista
        return [str(x).strip() for x in raw if str(x).strip()]
    s = str(raw).strip()
    if not s or s in ('{}','[]'):
        return []
    # remove chaves externas se houver
    if (s.startswith("{") and s.endswith("}")) or (s.startswith("[") and s.endswith("]")):
        s = s[1:-1]
    # agora parse com csv (respeita aspas)
    reader = csv.reader(io.StringIO(s))
    try:
        parts = next(reader, [])
    except Exception:
        # fallback simples
        parts = [p.strip().strip('"') for p in s.split(",")]
    # normalização final
    parts = [re.sub(r"\s+", " ", p).strip() for p in parts if p is not None and str(p).strip()]
    return parts

amen_list = df["amenities"].map(parse_amenities) if "amenities" in df.columns else pd.Series([[]]*len(df), index=df.index)
df["amenities_count"] = amen_list.map(len).astype("Int64")

key_flags = {
    "Wifi": "has_wifi",
    "Air conditioning": "has_ac",
    "Pool": "has_pool",
    "Kitchen": "has_kitchen",
    "Washer": "has_washer",
    "Dryer": "has_dryer",
    "Heating": "has_heating",
    "TV": "has_tv",
}
for label, colname in key_flags.items():
    df[colname] = amen_list.map(lambda xs: (label in xs) if isinstance(xs, list) else False).astype("boolean")

# =========================
# 6) Padronizações *_std
# =========================
room_map = {
    "Entire home/apt":"entire_home",
    "Private room":"private_room",
    "Shared room":"shared_room",
    "Hotel room":"hotel_room"
}
df["room_type_std"] = df["room_type"].map(room_map).fillna("other")

prop_map = {
    "Apartment":"apartment", "Condominium":"apartment", "Loft":"apartment",
    "House":"house", "Townhouse":"house", "Villa":"house",
    "Guest suite":"guest_space","Guesthouse":"guest_space",
}
df["property_type_std"] = df["property_type"].map(prop_map).fillna("other")

bed_map = {
    "Real Bed":"real_bed", "Futon":"sofa_bed", "Pull-out Sofa":"sofa_bed",
    "Couch":"sofa_bed", "Airbed":"airbed"
}
df["bed_type_std"] = df["bed_type"].map(bed_map).fillna("other")

# =========================
# 7) Auditoria
# =========================
df["fonte"] = "Airbnb_bronze"
# Se no banco dt_ingestao tem DEFAULT NOW(), recomendo não enviar essa coluna.
# Se quiser enviar do DF:
# df["dt_ingestao"] = pd.Timestamp.utcnow().to_pydatetime()

# =========================
# 8) Silver: manter apenas as colunas para importantes
#  =========================
keep_cols = [
    "fk_anuncio",
    "room_type_std","property_type_std","bed_type_std",
    "accommodates","bathrooms","bedrooms","beds",
    "amenities_count",
    "has_wifi","has_ac","has_pool","has_kitchen","has_washer","has_dryer","has_heating","has_tv",
    "has_availability","availability_30","availability_60","availability_90","availability_365",
    "minimum_nights","maximum_nights",
    "fonte"
    # "dt_ingestao"  # deixe de fora se usar DEFAULT do banco
]
df_propriedade_silver = df[keep_cols].copy()

# Sanity checks opcionais
assert df_propriedade_silver["fk_anuncio"].notna().all(), "Há fk_anuncio nulo."

In [7]:

def _to_python_scalar(v):
    # nulos (cobre NaN, NaT, <NA>)
    try:
        if v is None or pd.isna(v):
            return None
    except Exception:
        pass

    # pandas Timestamp -> datetime
    if isinstance(v, pd.Timestamp):
        return v.to_pydatetime()

    # numpy datetime64 -> datetime
    if isinstance(v, np.datetime64):
        return pd.to_datetime(v).to_pydatetime()

    # bool numpy/pandas -> bool
    if isinstance(v, (np.bool_, bool)):
        return bool(v)

    # int numpy -> int
    if isinstance(v, np.integer):
        return int(v)

    # float numpy -> float (cuidando de NaN já tratado acima)
    if isinstance(v, np.floating):
        fv = float(v)
        return None if np.isnan(fv) else fv

    # strings vazias / marcadores de nulo
    if isinstance(v, str) and v.strip().lower() in {"", "nan", "none", "null"}:
        return None

    return v

def df_sql_safe(df: pd.DataFrame) -> pd.DataFrame:
    df2 = df.copy()

    for c in df2.columns:
        # datetimes (com ou sem tz) -> datetime or None
        if pd.api.types.is_datetime64_any_dtype(df2[c]):
            # remove tz se existir, converte para objeto e aplica conversão
            s = df2[c]
            try:
                if hasattr(s.dt, "tz_localize"):
                    # se tiver tz, converte para naive
                    s = s.dt.tz_convert(None)
            except Exception:
                pass
            df2[c] = s.astype("object").map(_to_python_scalar)
            continue

        # boolean pandas (nullable) -> bool/None
        if pd.api.types.is_bool_dtype(df2[c]):
            df2[c] = df2[c].astype("object").map(_to_python_scalar)
            continue

        # inteiros "puros" (np.int64) -> int (Python)
        if pd.api.types.is_integer_dtype(df2[c]):
            # cuidado: isso perde o <NA> (use Int64 se precisar nulos). Se já não há nulos, ok.
            df2[c] = df2[c].astype("object").map(_to_python_scalar)
            continue

        # inteiros "nullable" (Int64) -> int/None
        if str(df2[c].dtype) == "Int64":
            df2[c] = df2[c].astype("object").map(_to_python_scalar)
            continue

        # float -> float/None
        if pd.api.types.is_float_dtype(df2[c]):
            df2[c] = df2[c].astype("object").map(_to_python_scalar)
            continue

        # objetos: normaliza "nan"/"none"/"null" -> None
        if pd.api.types.is_object_dtype(df2[c]) or pd.api.types.is_string_dtype(df2[c]):
            df2[c] = df2[c].map(_to_python_scalar)

    return df2

# === APLICA NO SEU DF FINAL ===
df_propriedade_silver_sql = df_sql_safe(df_propriedade_silver)


In [8]:
dups = df_propriedade_silver_sql['fk_anuncio'].duplicated(keep=False)
assert not dups.any(), f"Duplicatas no DF para fk_anuncio: {df_propriedade_silver_sql.loc[dups, 'fk_anuncio'].head()}"


AssertionError: Duplicatas no DF para fk_anuncio: 0    14063
1    17878
2    24480
3    25026
4    31560
Name: fk_anuncio, dtype: int64

In [20]:
dfi = df_propriedade_silver_sql.copy()

# 1) garanta que fk_anuncio não é nulo
dfi = dfi[dfi['fk_anuncio'].notna()].copy()

# 2) score de completude (quantos campos NÃO nulos)
cols_comp = [c for c in dfi.columns if c not in ('fk_anuncio')]
dfi['_nn'] = dfi[cols_comp].notna().sum(axis=1)

# 3) chaves de preferência
dfi['_amen']   = dfi['amenities_count'].fillna(-1)
dfi['_avail']  = dfi['has_availability'].fillna(False).astype(int)
dfi['_baths']  = dfi['bathrooms'].fillna(-1)
dfi['_beds']   = dfi['beds'].fillna(-1)
dfi['_bdrms']  = dfi['bedrooms'].fillna(-1)
dfi['_acc']    = dfi['accommodates'].fillna(-1)
# dt_ingestao pode não existir; cria se não tiver
if 'dt_ingestao' not in dfi.columns:
    dfi['dt_ingestao'] = pd.NaT

# 4) ordena pelo melhor candidato e deduplica
dfi_sorted = dfi.sort_values(
    by=[
        'fk_anuncio',
        '_nn',       # mais campos preenchidos
        '_amen',     # mais amenities
        '_avail',    # disponibilidade
        '_baths',    # mais banheiros
        '_bdrms',    # mais quartos
        '_beds',     # mais camas
        '_acc',      # mais capacidade
        'dt_ingestao' # mais recente
    ],
    ascending=[True, False, False, False, False, False, False, False, False]
)

dfi_dedup = (
    dfi_sorted
    .drop_duplicates(subset=['fk_anuncio'], keep='first')
    .drop(columns=['_nn','_amen','_avail','_baths','_beds','_bdrms','_acc'])
)

# 5) diagnóstico
n_before = len(dfi)
n_after  = len(dfi_dedup)
n_dups   = n_before - n_after
print(f"Removidas {n_dups} duplicatas de fk_anuncio (antes={n_before}, depois={n_after}).")

# 6) (opcional) sanity checks
assert not dfi_dedup['fk_anuncio'].duplicated().any(), "Ainda há duplicatas de fk_anuncio."


Removidas 832025 duplicatas de fk_anuncio (antes=902210, depois=70185).


In [22]:
TARGET_COLS = [ 
    "fk_anuncio",
    "room_type_std","property_type_std","bed_type_std",
    "accommodates","bathrooms","bedrooms","beds",
    "amenities_count",
    "has_wifi","has_ac","has_pool","has_kitchen","has_washer","has_dryer","has_heating","has_tv",
    "has_availability","availability_30","availability_60","availability_90","availability_365",
    "minimum_nights","maximum_nights",
    "fonte"
  # dt_ingestao fica de fora para usar DEFAULT
]

raw_conn.rollback()
upsert_df(
    conn=raw_conn,  
    df=dfi_dedup,
    schema="b_silver",
    table="T_DIM_PROPRIEDADE",
    target_cols=TARGET_COLS,
    conflict_cols=["fk_anuncio"],   # chave de UPSERT (PK/UNIQUE)
    # update_cols=None  -> atualiza todas as colunas exceto a de conflito (default)
    page_size=10000
)
print("✅ T_DIM_PROPRIEDADE carregada/atualizada no Silver.")


✅ T_DIM_PROPRIEDADE carregada/atualizada no Silver.


### **Tratamento tabela T_DIM_REGRAS**

In [14]:
df_regras = pd.read_sql('SELECT * FROM a_bronze."T_DIM_REGRAS"', conn)
df_regras.head(5)

,id_regras,fk_anuncio,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,1,14063,f,f,f,strict_14_with_grace_period,f,f,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,17878,f,t,f,strict,f,f,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,24480,f,f,f,strict,f,f,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,25026,f,f,f,strict,t,t,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,31560,f,t,f,strict,f,f,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
#Analise de colunas boleanas
cols_bool = [
    "requires_license",
    "instant_bookable",
    "is_business_travel_ready",
    "require_guest_profile_picture",
    "require_guest_phone_verification"
]

for col in cols_bool:
    print(f"{col}: {df_regras[col].unique()}")

requires_license: ['f']
instant_bookable: ['f' 't']
is_business_travel_ready: ['f']
require_guest_profile_picture: ['f' 't']
require_guest_phone_verification: ['f' 't']


In [16]:
#Copia do df para padronização
df = df_regras.copy()

#1) Padronização dos valores nulos
valores_nulos = ["NaN", "nan", "None", "none", "NULL", "null", ""]
colunas = df.select_dtypes(include="object").columns
for c in colunas:
    df[c] = df[c].replace(list(valores_nulos), np.nan)


#2) Higienização simples de strings
def clean_text(s):
    if not isinstance(s, str): return s
    s = s.strip()
    s = re.sub(r"\s+", " ", s) # colapsa espacos 
    s = "".join(ch for ch in s if ch >= " ") #Remover caracteres de controle \n \t etc..
    return s 

for c in colunas:
    df[c] = df[c].map(clean_text)


#3) Normalizar politica de cancelamento
policy_map = {
    "flexible": "flexible",
    "moderate": "moderate",
    "strict": "strict",
    "strict_14_with_grace_period": "strict_14_gp",
    "super_strict_30": "super_strict_30",
    "super_strict_60": "super_strict_60",
}
if "cancellation_policy" in df.columns:
    df["cancellation_policy"] = (
        df["cancellation_policy"]
        .str.strip().str.lower()
        .map(policy_map).astype("category")
    )

#4) Remover colunas >60% de valores nulos

protect = {"fk_anuncio","id_regras","cancellation_policy"}  # adicione outras
null_ratio = df.isnull().mean()
to_drop = [c for c in null_ratio.index if null_ratio[c] >= 0.6 and c not in protect]
df = df.drop(columns=to_drop, errors="ignore")


#8) Transformar booleanas t/f em True e False
bool_map = {"t": True, "f": False, True: True, False: False}
for c in ["requires_license","instant_bookable","is_business_travel_ready","require_guest_profile_picture",
          "require_guest_phone_verification"]:
    if c in df.columns:
        df[c] = df[c].map(bool_map).astype("bool")



#9) Auditoria
df["fonte"] = 'Airbnb_bronze'
df['dt_ingestao'] = pd.Timestamp.utcnow()

#Varre o DataFrame procurando colunas com dtype object que, na prática, só têm:
#números (1.0, 2.0, NaN → vira float64)
#inteiros (1, 2, NaN → vira Int64)
#booleanos (True, False, NaN → vira boolean)


#6) Checagens de qualidade (QA) antes de gravar
dups = df["fk_anuncio"].duplicated(keep=False).sum() if "fk_anuncio" in df.columns else 0
assert dups == 0, f"Há {dups} duplicatas de fk_anuncio em T_DIM_REGRAS"



df = df.infer_objects(copy=False)

#10) df com os ajustes realizados "Pronto para inserir no banco"
df_regras_silver = df.copy()


In [17]:
df_regras_silver

,id_regras,fk_anuncio,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,fonte,dt_ingestao
0,1,14063,False,False,False,strict_14_gp,False,False,1,Airbnb_bronze,2025-08-26 14:40:55.563003+00:00
1,2,17878,False,True,False,strict,False,False,1,Airbnb_bronze,2025-08-26 14:40:55.563003+00:00
2,3,24480,False,False,False,strict,False,False,1,Airbnb_bronze,2025-08-26 14:40:55.563003+00:00
3,4,25026,False,False,False,strict,True,True,3,Airbnb_bronze,2025-08-26 14:40:55.563003+00:00
4,5,31560,False,True,False,strict,False,False,1,Airbnb_bronze,2025-08-26 14:40:55.563003+00:00
...,...,...,...,...,...,...,...,...,...,...,...
70180,902169,38820744,False,False,False,strict_14_gp,False,False,1,Airbnb_bronze,2025-08-26 14:40:55.563003+00:00
70181,902187,38831236,False,False,False,strict_14_gp,False,True,4,Airbnb_bronze,2025-08-26 14:40:55.563003+00:00
70182,902189,38834691,False,False,False,strict_14_gp,False,False,4,Airbnb_bronze,2025-08-26 14:40:55.563003+00:00
70183,902190,38834816,False,True,False,strict_14_gp,False,False,1,Airbnb_bronze,2025-08-26 14:40:55.563003+00:00


#### Tipagem e QA para garantir a gravação no banco de dados

In [18]:
# booleans como pandas 'boolean' (aceita NaN)
cols_bool = [
    "requires_license","instant_bookable","is_business_travel_ready",
    "require_guest_profile_picture","require_guest_phone_verification"
]
for c in cols_bool:
    if c in df_regras_silver.columns:
        df_regras_silver[c] = df_regras_silver[c].astype("boolean")

# contagem como inteiro nullable
if "calculated_host_listings_count" in df_regras_silver.columns:
    df_regras_silver["calculated_host_listings_count"] = (
        pd.to_numeric(df_regras_silver["calculated_host_listings_count"], errors="coerce").astype("Int64")
    )

# política como categoria controlada
policy_map = {
    "flexible":"flexible","moderate":"moderate","strict":"strict","strict_14_gp":"strict_14_gp",
    "super_strict_30":"super_strict_30","super_strict_60":"super_strict_60"
}
df_regras_silver["cancellation_policy"] = (
    df_regras_silver["cancellation_policy"].str.lower().astype("category")
)


In [19]:
# unicidade por anúncio (se a regra do seu modelo é 1:1)
dups = df_regras_silver["fk_anuncio"].duplicated(keep=False).sum()
assert dups == 0, f"Duplicatas em fk_anuncio: {dups}"

# domínios válidos nas booleanas
for c in cols_bool:
    invalid = set(df_regras_silver[c].dropna().unique()) - {True, False}
    assert not invalid, f"{c} com valores inválidos: {invalid}"

# checagem de nulos em chaves
assert df_regras_silver["fk_anuncio"].notna().all(), "fk_anuncio com nulos"
assert df_regras_silver["id_regras"].notna().all(), "id_regras com nulos"

# sanity check de distribuição
pct_true = (df_regras_silver[cols_bool].mean() * 100).round(2)
print("%% True por coluna:\n", pct_true)
print("\nTop cancellation policies:\n", df_regras_silver["cancellation_policy"].value_counts().head())


%% True por coluna:
 requires_license                      0.0
instant_bookable                    43.16
is_business_travel_ready              0.0
require_guest_profile_picture         1.5
require_guest_phone_verification      1.6
dtype: Float64

Top cancellation policies:
 cancellation_policy
flexible           32602
strict             16200
moderate           10770
strict_14_gp        9871
super_strict_60      554
Name: count, dtype: int64


In [20]:
# exige qualquer verificação do hóspede?
df_regras_silver["requires_any_verification"] = (
    df_regras_silver["require_guest_profile_picture"].fillna(False) |
    df_regras_silver["require_guest_phone_verification"].fillna(False)
).astype("boolean")

# “rigidez” da política de cancelamento para métricas/ordenação
strict_score = {
    "flexible":0, "moderate":1, "strict":2, "strict_14_gp":3, "super_strict_30":4, "super_strict_60":5
}
df_regras_silver["cancellation_strictness"] = (
    df_regras_silver["cancellation_policy"].map(strict_score).astype("Int64")
)


### carga dos dados no banco

In [36]:
from pandas.api.types import (
    is_datetime64_any_dtype, is_datetime64tz_dtype,
    is_float_dtype, is_object_dtype
)


def sanitize_for_psycopg2(df: pd.DataFrame, bool_cols=(), int_cols=(), cat_cols=()):
    """
    Converte dtypes pandas/numpy para tipos nativos do Python (bool/int/float/str/datetime/None).
    - Datetimes tz-aware -> UTC naive (TIMESTAMP WITHOUT TIME ZONE)
    - BooleanDtype -> bool/None
    - Int64 (nullable) -> int/None
    - Category/String -> str/None
    """
    out = df.copy()

    # 1) Booleanas: bool/None
    for c in bool_cols:
        if c in out.columns:
            out[c] = (
                out[c].astype("boolean")     # garante dtype nullable
                      .astype(object)
                      .map(lambda x: None if pd.isna(x) else bool(x))
            )

    # 2) Inteiros nullable: int/None
    for c in int_cols:
        if c in out.columns:
            out[c] = pd.to_numeric(out[c], errors="coerce").astype("Int64")
            out[c] = out[c].astype(object).map(lambda x: None if pd.isna(x) else int(x))

    # 3) Categorias/strings: str/None
    for c in cat_cols:
        if c in out.columns:
            out[c] = out[c].astype("string")
            out[c] = out[c].astype(object).map(lambda x: None if pd.isna(x) else str(x))

    # 4) Datetimes: tratar tz-aware -> UTC naive; NaT -> None
    for c in out.columns:
        if is_datetime64tz_dtype(out[c]):
            # garante série datetime com tz (UTC) e remove tz
            out[c] = pd.to_datetime(out[c], utc=True)
            out[c] = out[c].dt.tz_convert("UTC").dt.tz_localize(None)
            out[c] = out[c].astype(object).map(lambda x: None if pd.isna(x) else x)
        elif is_datetime64_any_dtype(out[c]):
            out[c] = out[c].astype("datetime64[ns]")
            out[c] = out[c].astype(object).map(lambda x: None if pd.isna(x) else x)
        elif is_float_dtype(out[c]) or is_object_dtype(out[c]):
            # garantir None no lugar de NaN
            out[c] = out[c].where(out[c].notna(), None)

    return out

In [ ]:

TARGET_COLS = [ 
    'id_regras', 'fk_anuncio', 'requires_license', 'instant_bookable',
    'is_business_travel_ready', 'cancellation_policy',
    'require_guest_profile_picture', 'require_guest_phone_verification',
    'calculated_host_listings_count', 'fonte',
    'requires_any_verification', 'cancellation_strictness'
]
df_for_upsert = df_regras_silver.drop(columns=["dt_ingestao"], errors="ignore")

cols_bool = [
    "requires_license","instant_bookable","is_business_travel_ready",
    "require_guest_profile_picture","require_guest_phone_verification",
    "requires_any_verification"
]
cols_int = ["calculated_host_listings_count", "cancellation_strictness", "id_regras", "fk_anuncio"]
cols_cat = ["cancellation_policy", "fonte"]

df_up = sanitize_for_psycopg2(
    df_for_upsert[TARGET_COLS],
    bool_cols=cols_bool,
    int_cols=cols_int,
    cat_cols=cols_cat
)

raw_conn.rollback()
upsert_df(
    conn=raw_conn,
    df=df_up,
    schema="b_silver",
    table="T_DIM_REGRAS",
    target_cols=TARGET_COLS,
    conflict_cols=["fk_anuncio"],
    page_size=10000
)
print("✅ T_DIM_REGRAS carregada/atualizada no Silver.")

C:\Users\User\AppData\Local\Temp\ipykernel_14904\422615775.py:40: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(out[c]):
C:\Users\User\AppData\Local\Temp\ipykernel_14904\422615775.py:40: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(out[c]):
C:\Users\User\AppData\Local\Temp\ipykernel_14904\422615775.py:40: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(out[c]):


✅ T_DIM_REGRAS carregada/atualizada no Silver.


### **Tratamento tabela T_FATO_AVALIACAO**

In [12]:
#Leitura tabela avaliação
df_avaliacao = pd.read_sql('SELECT * FROM a_bronze."T_FATO_AVALIACAO"', conn)
df_avaliacao.head(5)

,id_reviews,fk_anuncio,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,number_of_reviews_ltm
0,1,14063,38.0,2010-01-03,2018-03-04,91.0,9.0,9.0,9.0,9.0,9.0,9.0,0.38,None
1,2,17878,205.0,2010-07-15,2018-04-10,93.0,10.0,9.0,10.0,10.0,9.0,9.0,2.17,None
2,3,24480,85.0,2010-09-26,2018-02-14,95.0,10.0,10.0,10.0,10.0,10.0,9.0,0.92,None
3,4,25026,208.0,2010-06-07,2018-03-28,94.0,9.0,10.0,10.0,10.0,10.0,9.0,2.17,None
4,5,31560,200.0,2010-07-11,2018-04-01,96.0,10.0,10.0,10.0,10.0,10.0,9.0,2.12,None


In [13]:
df_avaliacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 902210 entries, 0 to 902209
Data columns (total 14 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id_reviews                   902210 non-null  int64  
 1   fk_anuncio                   902210 non-null  int64  
 2   number_of_reviews            902210 non-null  float64
 3   first_review                 477922 non-null  object 
 4   last_review                  477961 non-null  object 
 5   review_scores_rating         454194 non-null  float64
 6   review_scores_accuracy       453624 non-null  float64
 7   review_scores_cleanliness    453797 non-null  float64
 8   review_scores_checkin        453560 non-null  float64
 9   review_scores_communication  453795 non-null  float64
 10  review_scores_location       453657 non-null  float64
 11  review_scores_value          453659 non-null  float64
 12  reviews_per_month            477922 non-null  float64
 13 

### **Tratamento tabela T_FATO_AVALIACAO**

In [25]:
df_avaliacao['number_of_reviews_ltm'].unique()

array([None, '20', '48', '30', '35', '0', '10', '18', '1', '9', '15',
       '23', '50', '64', '5', '12', '28', '8', '7', '14', '16', '6', '24',
       '2', '17', '19', '4', '13', '11', '36', '29', '62', '44', '3',
       '32', '40', '73', '27', '53', '31', '25', '22', '39', '66', '34',
       '21', '38', '47', '42', '37', '41', '51', '45', '56', '26', '33',
       '52', '58', '68', '60', '43', '55', '59', '46', '69', '63', '49',
       '71', '82', '57', '67', '54', '91', '98', '87', '74', '81', '100',
       '92', '61', '65', '89', '76', '107', '75', '95', '96', '84', '83',
       '88', '110', '77', '86', '104', '93', '80', '122', '70', '97',
       '72', '94', '78', '101', '103', '105', '119', '85', '108', '99',
       '79', '90', '127', '102'], dtype=object)

In [ ]:
##---------------------------------------------------
#Tratamento de colunas e criação de novas features
##---------------------------------------------------

df = df_avaliacao.copy()

##---------------------------------------------------
#1)Colunas de datas

col_obj = ['first_review','last_review']

for c in col_obj:
    df[c] = pd.to_datetime(df[c], errors='coerce')





##---------------------------------------------------
#2) colunas numericas 
col_num = ['number_of_reviews', 'number_of_reviews_ltm']

for c in col_num:
    df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0).astype(int)

#reviews_per_month -> float (nulo vira 0)
df["reviews_per_month"] = pd.to_numeric(df["reviews_per_month"], errors="coerce").fillna(0.0)

#Limpar outliers extremos de reviews_per_month (winsorize simples)
if df["reviews_per_month"].notna().any():
    p99 = df["reviews_per_month"].quantile(0.99)
    df["reviews_per_month"] = df["reviews_per_month"].clip(upper=p99)

##---------------------------------------------------
#3) normalização de estava (0-10) das notas review

score_cols = [
    "review_scores_rating","review_scores_accuracy","review_scores_cleanliness",
    "review_scores_checkin","review_scores_communication","review_scores_location",
    "review_scores_value"
]

def scale_0_10(s: pd.Series) -> pd.Series:
    """
    - Se a seria tiver valores  >10 assume a escala 0-100 e divide por 10.
    - clip 0 - 10 para limitar os limites de avaliação exemplo: avaliação 15 sera convertido para 10, 
    avaliação -5 sera convertido para 0.
    - Manten Nan onde tiver valores nulos.
    """
    s = pd.to_numeric(s, errors='coerce')
    if (s.dropna() >10).any():
        s = s / 10.0

    return s.clip(lower=0, upper=10)

for c in score_cols:
    df[f"{c}_0_10"] = scale_0_10(df[c])


##---------------------------------------------------
#4) Criação de novas features

# Duração em dias entre primeiro e último review
dur = (df["last_review"] - df["first_review"]).dt.days
df["duracao_reviews_dias"] = dur.where(dur >= 0, np.nan)

#Anos ativos (aproximado)
df['ativo_anos'] = (df['duracao_reviews_dias'] / 365.25).astype(float)

# Média de reviews por ano (evita divisão por zero)
df["reviews_por_ano"] = np.where(
    df["ativo_anos"] > 0,
    df["number_of_reviews"].astype(float) / df["ativo_anos"],
    np.nan
)

# Flag tem_review (1 se tem ao menos 1 review)
df["tem_review"] = (df["number_of_reviews"].fillna(0) > 0).astype("Int64")

# Nota média normalizada (0–10) usando as colunas já trazidas para 0–10
score_std_cols = [f"{c}_0_10" for c in score_cols]
df["nota_media_0_10"] = df[score_std_cols].mean(axis=1)

# Densidade de reviews dos últimos 12 meses
df["densidade_reviews_ltm"] = np.where(
    df["number_of_reviews"].astype(float) > 0,
    df["number_of_reviews_ltm"].astype(float) / df["number_of_reviews"].astype(float),
    np.nan
)



# ------------------------------------------------------------
# 5) Tipos finais e colunas de auditoria (Silver)
# ------------------------------------------------------------
# Converte ints "Int64" (pandas nullable) para int puro se desejar (cuidado com NaN).
# Aqui mantemos Int64 para preservar nulos com segurança.
int_cols_nullable = ["id_reviews","fk_anuncio","number_of_reviews","number_of_reviews_ltm","tem_review"]
for c in int_cols_nullable:
    df[c] = pd.to_numeric(df[c], errors="coerce").astype("Int64")

float_cols = ["reviews_per_month","nota_media_0_10","reviews_por_ano","ativo_anos","densidade_reviews_ltm"]
for c in float_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce").astype(float)

# Fonte e dt_ingestao (se quiser já levar para Silver)
df["fonte"] = "Airbnb_bronze"
df["dt_ingestao"] = pd.Timestamp.now(tz=None)  # deixe o banco aplicar default se preferir

# ------------------------------------------------------------
# 8) Seleção e ordenação final de colunas (prontas para a T_FATO_AVALIACAO Silver)
# ------------------------------------------------------------
final_cols = [
    "id_reviews","fk_anuncio",
    "number_of_reviews","number_of_reviews_ltm",
    "first_review","last_review","duracao_reviews_dias","ativo_anos","reviews_per_month","reviews_por_ano",
    "review_scores_rating_0_10","review_scores_accuracy_0_10","review_scores_cleanliness_0_10",
    "review_scores_checkin_0_10","review_scores_communication_0_10","review_scores_location_0_10",
    "review_scores_value_0_10","nota_media_0_10","densidade_reviews_ltm","tem_review",
    "fonte","dt_ingestao"
]

# Garante que todas existem; se algo faltar, cria nulo
for c in final_cols:
    if c not in df.columns:
        df[c] = np.nan

df_avaliacao_silver = df[final_cols].copy()




### Carga de dados para a tabela fato avaliação

In [34]:
df_avaliacao_silver.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 902210 entries, 0 to 902209
Data columns (total 22 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   id_reviews                        902210 non-null  Int64         
 1   fk_anuncio                        902210 non-null  Int64         
 2   number_of_reviews                 902210 non-null  Int64         
 3   number_of_reviews_ltm             902210 non-null  Int64         
 4   first_review                      477922 non-null  datetime64[ns]
 5   last_review                       477961 non-null  datetime64[ns]
 6   duracao_reviews_dias              477922 non-null  float64       
 7   ativo_anos                        477922 non-null  float64       
 8   reviews_per_month                 902210 non-null  float64       
 9   reviews_por_ano                   356094 non-null  float64       
 10  review_scores_rating_0_10       

In [41]:
TARGET_COLS = [ 
    "id_reviews","fk_anuncio",
    "number_of_reviews","number_of_reviews_ltm",
    "first_review","last_review","duracao_reviews_dias","ativo_anos","reviews_per_month","reviews_por_ano",
    "review_scores_rating_0_10","review_scores_accuracy_0_10","review_scores_cleanliness_0_10",
    "review_scores_checkin_0_10","review_scores_communication_0_10","review_scores_location_0_10",
    "review_scores_value_0_10","nota_media_0_10","densidade_reviews_ltm","tem_review",
    "fonte"
]

df_for_upsert = df_avaliacao_silver.drop(columns=["dt_ingestao"], errors="ignore")

cols_int = ["tem_review",'number_of_reviews_ltm', 'number_of_reviews','fk_anuncio','id_reviews']


date_cols = ["first_review", "last_review"]  # inclua outras se existirem
for c in date_cols:
    # vira objeto date do Python; NaT permanece NaT aqui…
    df_for_upsert[c] = df_for_upsert[c].dt.date
    # …então troque NaT por None (NULL para o banco)
    df_for_upsert[c] = df_for_upsert[c].astype("object").where(df_for_upsert[c].notna(), None)




df_up = sanitize_for_psycopg2(
    df_for_upsert[TARGET_COLS],
    int_cols=cols_int
)


raw_conn.rollback()
upsert_df(
    conn=raw_conn,
    df=df_up,
    schema="b_silver",
    table="T_FATO_AVALIACAO",
    target_cols=TARGET_COLS,
    conflict_cols=["id_reviews"],
    page_size=10000
)
print("✅ T_FATO_AVALIACAO carregada/atualizada no Silver.")

C:\Users\User\AppData\Local\Temp\ipykernel_17844\422615775.py:40: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(out[c]):
C:\Users\User\AppData\Local\Temp\ipykernel_17844\422615775.py:40: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(out[c]):
C:\Users\User\AppData\Local\Temp\ipykernel_17844\422615775.py:40: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(out[c]):
C:\Users\User\AppData\Local\Temp\ipykernel_17844\422615775.py:40: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz

✅ T_FATO_AVALIACAO carregada/atualizada no Silver.


### **Tratamento tabela T_FATO_PRECIFICACAO**

In [42]:
#Leitura tabela precificacao
df_preco = pd.read_sql('SELECT * FROM a_bronze. "T_FATO_PRECIFICACAO"', conn)
df_preco.head()

,id_preco,fk_anuncio,price,security_deposit,cleaning_fee,guests_included,extra_people,last_scraped,calendar_last_scraped
0,1,14063,133.0,1025.0,102.0,2,34.0,2018-04-14,2018-04-14
1,2,17878,270.0,0.0,273.0,2,51.0,2018-04-14,2018-04-14
2,3,24480,222.0,851.0,211.0,2,215.0,2018-04-14,2018-04-14
3,4,25026,161.0,1000.0,220.0,2,45.0,2018-04-14,2018-04-14
4,5,31560,222.0,NaN,85.0,2,68.0,2018-04-14,2018-04-14


In [46]:

df = df_preco.copy()

# -----------------------------
# 1) Datas (DATE no Postgres)
# -----------------------------
date_cols = ["last_scraped", "calendar_last_scraped"]
for c in date_cols:
    df[c] = pd.to_datetime(df[c], errors="coerce").dt.date
    df[c] = df[c].astype("object").where(pd.notna(df[c]), None)  # NaT -> None

# -----------------------------
# 2) Numéricos
# -----------------------------
money_cols = ["price", "security_deposit", "cleaning_fee", "extra_people"]
for c in money_cols:
    # Se já vier numérico, ok; se vier com símbolos, isso limpa.
    df[c] = (
        df[c]
        .astype(str)
        .str.replace(r"[^\d\.\-]", "", regex=True)
        .replace({"": np.nan, ".": np.nan})
    )
    df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0.0).clip(lower=0)

# Zero mínimos e evita negativos (dados "sujos")
for c in money_cols:
    df[c] = df[c].clip(lower=0)

# guests_included: inteiro (nullable)
df["guests_included"] = pd.to_numeric(df["guests_included"], errors="coerce").astype("Int64")

# -----------------------------
# 3) Features leves e úteis
# -----------------------------
def safe_div(num, den):
    num = pd.to_numeric(num, errors="coerce")
    den = pd.to_numeric(den, errors="coerce")
    return np.where((den > 0) & np.isfinite(den), num / den, np.nan)

# flags
df["has_cleaning_fee"] = (df["cleaning_fee"].fillna(0) > 0).astype("Int64")
df["has_deposit"]      = (df["security_deposit"].fillna(0) > 0).astype("Int64")

# razões (relativas ao preço)
df["cleaning_fee_ratio"] = safe_div(df["cleaning_fee"], df["price"])
df["deposit_ratio"]      = safe_div(df["security_deposit"], df["price"])

# preço por hóspede "incluído"
df["price_per_included_guest"] = np.where(
    df["guests_included"].fillna(0) > 0,
    df["price"] / df["guests_included"].astype(float),
    np.nan
)

# preço efetivo por noite para 2 e 4 hóspedes (sem amortizar cleaning_fee)
def nightly_for_party(price, guests_included, extra_people, party_size: int):
    gi = np.where(pd.isna(guests_included), 1, guests_included.astype(float))
    extra = np.maximum(party_size - gi, 0)
    total = price + extra_people * extra  # sem cleaning_fee (one-time)
    return total

df["price_2p_nightly"] = nightly_for_party(df["price"], df["guests_included"], df["extra_people"], 2)
df["price_4p_nightly"] = nightly_for_party(df["price"], df["guests_included"], df["extra_people"], 4)

# -----------------------------
# 4) Auditoria (Silver)
# -----------------------------
df["fonte"] = "Airbnb_bronze"
df["dt_ingestao"] = pd.Timestamp.now(tz=None)

# -----------------------------
# 5) Seleção final
# -----------------------------
final_cols = [
    "id_preco","fk_anuncio",
    "price","security_deposit","cleaning_fee","guests_included","extra_people",
    "last_scraped","calendar_last_scraped",
    "has_cleaning_fee","has_deposit","cleaning_fee_ratio","deposit_ratio",
    "price_per_included_guest","price_2p_nightly","price_4p_nightly",
    "fonte","dt_ingestao"
]
for c in final_cols:
    if c not in df.columns: df[c] = np.nan

df_precificacao_silver = df[final_cols].copy()


In [47]:
TARGET_COLS = [
    "id_preco","fk_anuncio",
    "price","security_deposit","cleaning_fee","guests_included","extra_people",
    "last_scraped","calendar_last_scraped",
    "has_cleaning_fee","has_deposit","cleaning_fee_ratio","deposit_ratio",
    "price_per_included_guest","price_2p_nightly","price_4p_nightly",
    "fonte"
]

df_for_upsert = df_precificacao_silver.drop(columns=["dt_ingestao"], errors="ignore").copy()

# Inteiros
cols_int = ["id_preco","fk_anuncio","guests_included","has_cleaning_fee","has_deposit"]

# (Datas já estão como date/None no ETL acima)
df_up = sanitize_for_psycopg2(
    df_for_upsert[TARGET_COLS],
    int_cols=cols_int
)

raw_conn.rollback()
upsert_df(
    conn=raw_conn,
    df=df_up,
    schema="b_silver",
    table="T_FATO_PRECIFICACAO",
    target_cols=TARGET_COLS,
    conflict_cols=["id_preco"],
    page_size=10000
)
print("✅ T_FATO_PRECIFICACAO carregada/atualizada no Silver.")


C:\Users\User\AppData\Local\Temp\ipykernel_17844\422615775.py:40: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(out[c]):
C:\Users\User\AppData\Local\Temp\ipykernel_17844\422615775.py:40: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(out[c]):
C:\Users\User\AppData\Local\Temp\ipykernel_17844\422615775.py:40: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(out[c]):
C:\Users\User\AppData\Local\Temp\ipykernel_17844\422615775.py:40: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz

✅ T_FATO_PRECIFICACAO carregada/atualizada no Silver.
